In [1]:
from openrec.tf2.metrics.ranking_metrics import Recall, NDCG, AUC

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.3.2'

In [4]:
import pandas as pd
import numpy as np

df_pred = pd.read_csv('./pred.csv')
df_test = pd.read_csv('./test.csv')

df_pred_new = pd.merge(
    df_pred, 
    df_test.loc[:, ['user_id', 'item_id', 'relevance']], on=['user_id', 'item_id'], how='left'
)

df_pred_new = df_pred_new[df_pred_new.user_id.isin(df_test.user_id)]

df_pred_new.fillna(0, inplace=True)

df_pred_new.loc[df_pred_new.relevance != 0, 'relevance'] = 1

df_pred_new.relevance.value_counts()

0.0    51516
1.0     3144
Name: relevance, dtype: int64

In [114]:
all_items = np.unique(np.hstack(
    [df_pred_new.item_id.unique(), df_test.item_id.unique()]
))

In [117]:
all_users = df_test.user_id.unique()

In [384]:
user_dict = dict(zip(all_users, range(len(all_users))))
item_dict = dict(zip(all_items, range(len(all_items))))

In [385]:
user_pred = np.zeros((len(all_users), len(all_items)))
user_pos_mask = np.zeros((len(all_users), len(all_items)), dtype=bool)

for user_id, item_id, score in zip(df_pred_new.user_id, df_pred_new.item_id, df_pred_new.score):
    user_pred[user_dict[user_id]][item_dict[item_id]] = score

for user_id, item_id, score in zip(df_test.user_id, df_test.item_id, df_test.relevance):
    user_pos_mask[user_dict[user_id]][item_dict[item_id]] = True

In [386]:
arr_inds = user_pred.argsort(axis=1)
arr_inds = np.flip(arr_inds, axis=1)

In [389]:
user_pred = np.take_along_axis(user_pred, arr_inds, axis=1)
user_pos_mask = np.take_along_axis(user_pos_mask, arr_inds, axis=1)

In [392]:
user_excl_mask = tf.constant(np.zeros_like(user_pos_mask, dtype=bool))
user_pred = tf.constant(user_pred.astype(np.float32), dtype=tf.float32)
user_pos_mask = tf.constant(user_pos_mask, dtype=tf.bool)

In [394]:
tf.math.reduce_mean(
    Recall(pos_mask=user_pos_mask, pred=user_pred, excl_mask=user_excl_mask, at=[20])
)

<tf.Tensor: shape=(), dtype=float32, numpy=0.096321315>

In [393]:
tf.math.reduce_mean(
    NDCG(pos_mask=user_pos_mask, pred=user_pred, excl_mask=user_excl_mask, at=[20])
)

<tf.Tensor: shape=(), dtype=float32, numpy=0.4626283>

In [414]:
tmp = AUC(pos_mask=user_pos_mask[:, :20], pred=user_pred[:, :20], excl_mask=user_excl_mask[:, :20])
tmp = tf.where(tf.math.is_nan(tmp), tf.zeros_like(tmp), tmp)
tf.math.reduce_mean(tmp)

<tf.Tensor: shape=(), dtype=float32, numpy=0.28317448>

In [466]:
from openrec.tf1.utils.evaluators import *

In [420]:
tmp = np.array(user_pos_mask)

In [450]:
tmp_ = np.array(
    pd.DataFrame(np.vstack(np.where(tmp == True)).T, columns=['row', 'col']).groupby('row')['col'].apply(np.array)
)

In [467]:
Precision(precision_at=[2]).compute(tmp_, None)

ValueError: non-broadcastable output operand with shape (1,) doesn't match the broadcast shape (7,)